# Example to load newly trained model

In [ ]:
import os
import sys

### path to PYSEQM ###
sys.path.insert(1, '.../PYSEQM_dev/')
import torch

### path to HIPNN ###
sys.path.append('.../hippynn/')

from seqm.seqm_functions.constants import Constants
from seqm.MolecularDynamics import Molecular_Dynamics_Basic, Molecular_Dynamics_Langevin
from seqm.MolecularDynamics import XL_BOMD
from seqm.MolecularDynamics import KSA_XL_BOMD
from hippynn.interfaces.pyseqm_interface.gen_par import gen_par
from hippynn.interfaces.pyseqm_interface.gen_par_full_model import gen_par_full_model

from seqm.Molecule import Molecule
from seqm.ElectronicStructure import Electronic_Structure

torch.set_default_dtype(torch.float64)
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

## MD

In [ ]:
species = torch.as_tensor([[8, 6, 1, 1],
                           [8, 6, 1, 1]], dtype=torch.int64, device=device)

coordinates = torch.tensor(
    [
        [[0.014,  0.001,  0.001],
         [1.336,  0.001,  0.001],
         [1.757,  1.039, -0.001],
         [1.757, -1.039,  0.000]],
        
        [[0.014,  0.001,  0.001],
         [1.336,  0.001,  0.001],
         [1.757,  1.039, -0.001],
         [1.757, -1.039,  0.000]],
    ],
    device=device,
)

const = Constants().to(device)

### ML part ###
model_file = "./TEST1/experiment_structure.pt" ###
state_file = "./TEST1/best_checkpoint.pt" ###
par_atom_node_name = "SEQM_Atom_Params"
seqm_node_name = "SEQM_Energy"

### This one goes to MD driver and generates HIPNN-SEQM parameters in dynamics
leared_par_generator = gen_par(model_file, state_file, par_atom_node_name, seqm_node_name, device)

from hippynn.experiment.serialization import load_checkpoint
model = load_checkpoint(model_file, state_file)["training_modules"][0].to(device)
seqm_parameters0 = model.node_from_name(seqm_node_name).torch_module.energy.seqm_parameters
del model

seqm_parameters = {
    "method": 'PM3', # seqm_parameters0["method"],  # AM1, MNDO, PM#
    "scf_eps": 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
    "scf_converger": [0, 0.1],  # converger used for scf loop
                                # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                # [1], adaptive mixing
                                # [2], adaptive mixing, then pulay
    "sp2": [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
    "elements": seqm_parameters0["elements"],  # [0,1,6,8],
    "learned": seqm_parameters0["learned"],  # learned parameters name list, e.g ['U_ss']
    #"parameter_file_dir": '.../PYSEQM_dev/seqm/params/',  # file directory for other required parameters
    "pair_outer_cutoff": 1.0e10,  # consistent with the unit on coordinates
    'eig' : True,
    'UHF' : False,
}


### Pass charges and multiplicity if 'UHF':True. Otherwise, neutral singlet is the default setting
### XL-BOMD supports only closed shell systems at the moment!

# charges = torch.tensor([0,0],dtype=torch.int64, device=device)
# mult = torch.tensor([1,3], device=device)
# molecule = Molecule(const, seqm_parameters, coordinates, species, charges, mult).to(device)

molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

### BOMD, NVE

In [ ]:
dt = 0.1

# create MD object and output files
md0 =  Molecular_Dynamics_Basic(seqm_parameters=seqm_parameters, Temp=400.0, timestep=dt).to(device)

# initialize velocities with predefined randomization seed.
torch.manual_seed(0)
md0.initialize_velocity(molecule )

# run
_ = md0.run(molecule, 5, remove_com=[True, 1], Info_log=False, learned_parameters=leared_par_generator)

### XL-BOMD, NVE

In [ ]:
### XL-BOMD supports only closed shell systems at the moment!

coordinates = torch.tensor(
    [
        [[0.014,  0.001,  0.001],
         [1.336,  0.001,  0.001],
         [1.757,  1.039, -0.001],
         [1.757, -1.039,  0.000]],
        
        [[0.014,  0.001,  0.001],
         [1.336,  0.001,  0.001],
         [1.757,  1.039, -0.001],
         [1.757, -1.039,  0.000]],
    ],
    device=device,
)

### Pass charges if 'UHF':True. Otherwise, neutral singlet is the default setting
### XL-BOMD supports only closed shell systems at the moment!

#charges = torch.tensor([0,2],dtype=torch.int64, device=device)
#molecule = Molecule(const, seqm_parameters, coordinates, species, charges).to(device)


molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

dt = 0.1

xl_bomd_params={'k':6}

md2 =  XL_BOMD(xl_bomd_params=xl_bomd_params,
              seqm_parameters=seqm_parameters, Temp=400.0, timestep=dt).to(device)

species = torch.as_tensor([[8, 6, 1, 1],
                           [8, 6, 1, 1]], dtype=torch.int64, device=device)


torch.manual_seed(0)
md2.initialize_velocity(molecule )
_ = md2.run(molecule, 5, remove_com=[True, 1], Info_log=False, learned_parameters=leared_par_generator)

## Single point

In [ ]:
### SINGLE POINT ###
####################


### create molecule object:
species = torch.as_tensor([[8,6,1,1],
                           [8,6,1,1],
                           [8,8,6,0]], # zero-padding for batching
                          dtype=torch.int64, device=device)

coordinates = torch.tensor([
                              [
                               [0.00,    0.00,    0.00],
                               [1.22,    0.00,    0.00],
                               [1.82,    0.94,    0.00],
                               [1.82,   -0.94,    0.00]
                              ],
                              [
                               [0.00,    0.00,    0.00],
                               [1.22,    0.00,    0.00],
                               [1.82,    0.94,    0.00],
                               [1.82,   -0.94,    0.00]
                              ],
                              [
                               [0.00,    0.00,    0.00],
                               [1.23,    0.00,    0.00],
                               [1.82,    0.94,    0.00],
                               [0.0,0.0,0.0]            # zero-padding for batching
                              ]
                            ], device=device)

const = Constants().to(device)

elements = [0]+sorted(set(species.reshape(-1).tolist()))

seqm_parameters = {
    "method": 'PM3', #seqm_parameters0["method"],  # AM1, MNDO, PM#
    "scf_eps": 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
    "scf_converger": [1, 0.0],  # converger used for scf loop
    "sp2": [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
    "elements": seqm_parameters0["elements"],  # [0,1,6,8],
    "learned": seqm_parameters0["learned"],  # learned parameters name list, e.g ['U_ss']
    #"parameter_file_dir": '.../PYSEQM_dev/seqm/params/',  # file directory for other required parameters
    "pair_outer_cutoff": 1.0e10,  # consistent with the unit on coordinates
    'eig' : True,
    'UHF' : False,
}

molecules = Molecule(const, seqm_parameters, coordinates, species).to(device)

### Create electronic structure driver:
esdriver = Electronic_Structure(seqm_parameters).to(device)

### Run esdriver on molecules:
esdriver(molecules,learned_parameters=leared_par_generator)

In [ ]:
molecules.Etot

In [ ]:
molecules.force